In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import seaborn as sns


%matplotlib inline

import matplotlib
%config InlineBackend.figure_format='retina'

from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import fcluster

# turn off warnings
import warnings
warnings.filterwarnings('ignore')
# turn off future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_excel('../CWTS Leiden Ranking Open Edition 2023 - Results.xlsx', sheet_name='Results')
df['Period'] = df['Period'].str.replace('\u2013', '\u002d')

In [3]:
# from df select rows with 'All sciences' in 'Field', '1' in 'Frac_counting', '2018-2021' in 'Period'
df['Period'] = df['Period'].str.replace('\u2013', '\u002d')
#df_sel = df[(df['Field'] == 'All sciences') & (df['Frac_counting'] == '1') & (df['Period'] == '2018-2021')]
df_sel = df[(df['Field'] == 'All sciences')]
df_sel = df_sel[(df_sel['Frac_counting'] == 1)]
df_sel = df_sel[(df_sel['Period'] == '2018-2021')]



In [4]:
df_sel['Country'].unique()

array(['Denmark', 'Finland', 'Iran', 'India', 'Poland', 'Ethiopia',
       'Greece', 'China', 'Egypt', 'France', 'South Korea', 'Turkey',
       'United States', 'Lebanon', 'Israel', 'United Kingdom',
       'New Zealand', 'Chile', 'Australia', 'Mexico', 'Romania',
       'Pakistan', 'Indonesia', 'Germany', 'Italy', 'Czech Republic',
       'Canada', 'Hungary', 'Spain', 'Sweden', 'Taiwan', 'Thailand',
       'Japan', 'Slovakia', 'Netherlands', 'Ireland', 'Viet Nam',
       'Cyprus', 'Switzerland', 'Brazil', 'Belgium', 'Austria', 'Qatar',
       'Russia', 'Saudi Arabia', 'Lithuania', 'United Arab Emirates',
       'Kuwait', 'Ghana', 'Uganda', 'Morocco', 'Singapore', 'Argentina',
       'Kazakhstan', 'South Africa', 'Norway', 'Oman', 'Estonia',
       'Uruguay', 'Colombia', 'Portugal', 'Tunisia', 'Algeria',
       'Malaysia', 'Serbia', 'Nigeria', 'Iceland', 'Jordan', 'Slovenia',
       'Luxembourg', 'Croatia', 'Cameroon'], dtype=object)

In [3]:
countries = ['Austria',
 'Croatia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Italy',
 'Latvia',
 'Lithuania',
 'Netherlands',
 'Poland',
 'Portugal',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden']

In [9]:
# check if all countries are present
for country in countries:
    if country not in df_sel['Country'].unique():
        print(country)

Latvia


In [10]:
df_autonomy = df_sel[df_sel['Country'].isin(countries)]

In [35]:
df_autonomy

,University,ROR ID,Country,Field,Period,Frac_counting,impact_P,collab_P,oa_P,TCS,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
25,Aalborg University,04m5j1k67,Denmark,All sciences,2018-2021,1,5447.096005,12500.0,12500.0,43907.135800,...,0.094320,0.079440,0.074240,0.084160,0.261600,0.253920,0.269600,0.0,0.0,0.0
181,Aalto University,020hwjq30,Finland,All sciences,2018-2021,1,4103.311991,9287.0,9287.0,37563.518978,...,0.232906,0.042856,0.038872,0.046840,0.337999,0.328739,0.346829,0.0,0.0,0.0
337,Aarhus University,01aj84f44,Denmark,All sciences,2018-2021,1,9417.490922,21868.0,21868.0,76375.783953,...,0.132385,0.099872,0.096214,0.104125,0.201527,0.196269,0.206512,0.0,0.0,0.0
493,Åbo Akademi University,029pk6x14,Finland,All sciences,2018-2021,1,1141.395043,3284.0,3284.0,8439.051381,...,0.255481,0.056029,0.048417,0.064251,0.140682,0.128502,0.152253,0.0,0.0,0.0
961,Adam Mickiewicz University in Poznań,04g6bbq64,Poland,All sciences,2018-2021,1,2552.839107,5069.0,5069.0,14110.247941,...,0.183665,0.043796,0.038272,0.050109,0.107122,0.098442,0.115013,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210469,Warsaw University of Technology,00y0xnp53,Poland,All sciences,2018-2021,1,3305.201168,5768.0,5768.0,13106.650324,...,0.129334,0.025485,0.021671,0.029473,0.069868,0.062587,0.076630,0.0,0.0,0.0
211717,"West Pomeranian University of Technology, Szcz...",0596m7f19,Poland,All sciences,2018-2021,1,1379.715381,2110.0,2110.0,7663.357275,...,0.098578,0.015640,0.010427,0.021327,0.025118,0.019431,0.031280,0.0,0.0,0.0
212653,Wrocław Medical University,01qpw1b93,Poland,All sciences,2018-2021,1,1747.678736,3667.0,3667.0,11524.523705,...,0.122989,0.073902,0.065721,0.082356,0.054268,0.047178,0.061085,0.0,0.0,0.0
212809,Wroclaw University of Environmental and Life S...,05cs8k179,Poland,All sciences,2018-2021,1,1183.573870,2229.0,2229.0,7590.524549,...,0.140422,0.013010,0.008973,0.017945,0.025123,0.018843,0.031853,0.0,0.0,0.0


In [11]:
# group df_autonomy by 'Country', calculate sum
df_autonomy_grouped = df_autonomy.groupby('Country').sum()

In [12]:
df_autonomy_grouped

,Frac_counting,impact_P,collab_P,oa_P,TCS,TNCS,P_top1,P_top5,P_top10,P_top50,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
Country,,,,,,,,,,,,,,,,,,,,,
Austria,12,30877.326659,75660.0,75660.0,2.371235e+05,33766.117163,337.433634,1706.754327,3393.655675,16453.813474,...,3.287543,0.795378,0.717059,0.876972,1.372329,1.270999,1.474441,0.0,0.0,0.0
Croatia,2,5986.611323,11517.0,11517.0,3.194233e+04,4988.182768,38.418797,207.625946,437.559517,2692.534282,...,0.133993,0.110920,0.097179,0.125633,0.199399,0.182083,0.218336,0.0,0.0,0.0
Denmark,5,43050.329367,100545.0,100545.0,3.782910e+05,49594.819471,535.641093,2635.441547,5088.218504,24266.017002,...,0.628293,0.472749,0.452271,0.493900,1.291120,1.259515,1.322351,0.0,0.0,0.0
Estonia,2,3598.945009,8013.0,8013.0,2.513510e+04,3357.111869,36.885050,149.604150,304.021451,1637.329006,...,0.217987,0.109692,0.095521,0.124280,0.284211,0.262539,0.306799,0.0,0.0,0.0
Finland,9,32650.975735,77396.0,77396.0,2.469200e+05,35433.795954,342.994126,1714.147940,3506.228107,17797.613219,...,2.110513,0.580024,0.530264,0.630310,2.146574,2.056064,2.236693,0.0,0.0,0.0
France,33,92574.423128,443100.0,443100.0,7.203223e+05,97348.381623,957.400275,4873.172972,9903.910257,48682.581948,...,6.350634,2.984931,2.792103,3.184566,7.901160,7.618063,8.190379,0.0,0.0,0.0
Italy,47,188947.396006,465733.0,465733.0,1.426604e+06,198062.836740,1861.375609,10004.323650,20632.396392,100322.731436,...,4.488942,3.741625,3.461329,4.025905,6.803010,6.447539,7.168939,0.0,0.0,0.0
Lithuania,3,4116.356437,7803.0,7803.0,2.141795e+04,3083.654505,17.393771,106.546469,259.336147,1833.472492,...,0.227000,0.154652,0.130941,0.180043,0.230246,0.199039,0.261209,0.0,0.0,0.0
Netherlands,13,94048.241778,243511.0,243511.0,9.077341e+05,124085.809469,1513.731909,7074.503141,13750.109790,56954.532275,...,4.124454,1.027478,0.976141,1.080858,2.699879,2.617589,2.783221,0.0,0.0,0.0


In [13]:
# save df_autonomy_grouped to excel
#df_autonomy_grouped.to_excel('leiden-open-ranking-countries-sum.xlsx')


In [6]:
df_leiden = pd.read_excel('leiden-open-ranking-countries-sum.xlsx')
df_leiden

,Country,Frac_counting,impact_P,collab_P,oa_P,TCS,TNCS,P_top1,P_top5,P_top10,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
0,Austria,12,30877.326659,75660,75660,2.371235e+05,33766.117163,337.433634,1706.754327,3393.655675,...,3.287543,0.795378,0.717059,0.876972,1.372329,1.270999,1.474441,0,0,0
1,Croatia,2,5986.611323,11517,11517,3.194233e+04,4988.182768,38.418797,207.625946,437.559517,...,0.133993,0.110920,0.097179,0.125633,0.199399,0.182083,0.218336,0,0,0
2,Denmark,5,43050.329367,100545,100545,3.782910e+05,49594.819471,535.641093,2635.441547,5088.218504,...,0.628293,0.472749,0.452271,0.493900,1.291120,1.259515,1.322351,0,0,0
3,Estonia,2,3598.945009,8013,8013,2.513510e+04,3357.111869,36.885050,149.604150,304.021451,...,0.217987,0.109692,0.095521,0.124280,0.284211,0.262539,0.306799,0,0,0
4,Finland,9,32650.975735,77396,77396,2.469200e+05,35433.795954,342.994126,1714.147940,3506.228107,...,2.110513,0.580024,0.530264,0.630310,2.146574,2.056064,2.236693,0,0,0
5,France,33,92574.423128,443100,443100,7.203223e+05,97348.381623,957.400275,4873.172972,9903.910257,...,6.350634,2.984931,2.792103,3.184566,7.901160,7.618063,8.190379,0,0,0
6,Italy,47,188947.396006,465733,465733,1.426604e+06,198062.836740,1861.375609,10004.323650,20632.396392,...,4.488942,3.741625,3.461329,4.025905,6.803010,6.447539,7.168939,0,0,0
7,Lithuania,3,4116.356437,7803,7803,2.141795e+04,3083.654505,17.393771,106.546469,259.336147,...,0.227000,0.154652,0.130941,0.180043,0.230246,0.199039,0.261209,0,0,0
8,Netherlands,13,94048.241778,243511,243511,9.077341e+05,124085.809469,1513.731909,7074.503141,13750.109790,...,4.124454,1.027478,0.976141,1.080858,2.699879,2.617589,2.783221,0,0,0
9,Poland,37,70775.141611,132821,132821,3.928681e+05,57552.084893,438.782894,2391.713409,5062.043034,...,4.970763,1.286096,1.063639,1.518935,2.075163,1.802613,2.358964,0,0,0


In [66]:
# group df_autonomy by 'Country', calculate mean and std
df_autonomy_grouped_mean = df_autonomy.groupby('Country').mean()
df_autonomy_grouped_std = df_autonomy.groupby('Country').std()


In [67]:
df_autonomy_grouped_mean

,Frac_counting,impact_P,collab_P,oa_P,TCS,TNCS,P_top1,P_top5,P_top10,P_top50,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
Country,,,,,,,,,,,,,,,,,,,,,
Austria,1.0,2573.110555,6305.000000,6305.000000,19760.288801,2813.843097,28.119469,142.229527,282.804640,1371.151123,...,0.273962,0.066281,0.059755,0.073081,0.114361,0.105917,0.122870,0.0,0.0,0.0
Croatia,1.0,2993.305662,5758.500000,5758.500000,15971.163500,2494.091384,19.209399,103.812973,218.779758,1346.267141,...,0.066996,0.055460,0.048589,0.062816,0.099699,0.091042,0.109168,0.0,0.0,0.0
Denmark,1.0,8610.065873,20109.000000,20109.000000,75658.197088,9918.963894,107.128219,527.088309,1017.643701,4853.203400,...,0.125659,0.094550,0.090454,0.098780,0.258224,0.251903,0.264470,0.0,0.0,0.0
Estonia,1.0,1799.472504,4006.500000,4006.500000,12567.549993,1678.555934,18.442525,74.802075,152.010726,818.664503,...,0.108994,0.054846,0.047761,0.062140,0.142106,0.131269,0.153399,0.0,0.0,0.0
Finland,1.0,3627.886193,8599.555556,8599.555556,27435.551984,3937.088439,38.110458,190.460882,389.580901,1977.512580,...,0.234501,0.064447,0.058918,0.070034,0.238508,0.228452,0.248521,0.0,0.0,0.0
France,1.0,2805.285549,13427.272727,13427.272727,21827.947620,2949.950958,29.012130,147.671908,300.118493,1475.229756,...,0.192443,0.090452,0.084609,0.096502,0.239429,0.230850,0.248193,0.0,0.0,0.0
Italy,1.0,4020.157362,9909.212766,9909.212766,30353.283284,4214.102909,39.603736,212.857950,438.987157,2134.526201,...,0.095509,0.079609,0.073645,0.085658,0.144745,0.137182,0.152531,0.0,0.0,0.0
Lithuania,1.0,1372.118812,2601.000000,2601.000000,7139.317724,1027.884835,5.797924,35.515490,86.445382,611.157497,...,0.075667,0.051551,0.043647,0.060014,0.076749,0.066346,0.087070,0.0,0.0,0.0
Netherlands,1.0,7234.480137,18731.615385,18731.615385,69825.702521,9545.062267,116.440916,544.192549,1057.700753,4381.117867,...,0.317266,0.079037,0.075088,0.083143,0.207683,0.201353,0.214094,0.0,0.0,0.0


In [53]:
df_autonomy_grouped.columns

Index(['Frac_counting', 'impact_P', 'collab_P', 'oa_P', 'TCS', 'TNCS',
       'P_top1', 'P_top5', 'P_top10', 'P_top50', 'P_collab', 'P_int_collab',
       'P_industry_collab', 'P_short_dist_collab', 'P_long_dist_collab',
       'P_OA', 'P_gold_OA', 'P_hybrid_OA', 'P_bronze_OA', 'P_green_OA',
       'P_OA_unknown', 'MCS', 'MCS_lb', 'MCS_ub', 'MNCS', 'MNCS_lb', 'MNCS_ub',
       'PP_top1', 'PP_top1_lb', 'PP_top1_ub', 'PP_top5', 'PP_top5_lb',
       'PP_top5_ub', 'PP_top10', 'PP_top10_lb', 'PP_top10_ub', 'PP_top50',
       'PP_top50_lb', 'PP_top50_ub', 'PP_collab', 'PP_collab_lb',
       'PP_collab_ub', 'PP_int_collab', 'PP_int_collab_lb', 'PP_int_collab_ub',
       'PP_industry_collab', 'PP_industry_collab_lb', 'PP_industry_collab_ub',
       'PP_short_dist_collab', 'PP_short_dist_collab_lb',
       'PP_short_dist_collab_ub', 'PP_long_dist_collab',
       'PP_long_dist_collab_lb', 'PP_long_dist_collab_ub', 'PP_OA', 'PP_OA_lb',
       'PP_OA_ub', 'PP_gold_OA', 'PP_gold_OA_lb', 'PP_gold_OA

In [68]:
df_autonomy_grouped_mean[['P_top10', 'P_top1']]

,P_top10,P_top1
Country,,
Austria,282.804640,28.119469
Croatia,218.779758,19.209399
Denmark,1017.643701,107.128219
Estonia,152.010726,18.442525
Finland,389.580901,38.110458
France,300.118493,29.012130
Italy,438.987157,39.603736
Lithuania,86.445382,5.797924
Netherlands,1057.700753,116.440916


In [69]:
df_autonomy_grouped_std[['P_top10', 'P_top1']]

,P_top10,P_top1
Country,,
Austria,182.345157,20.056623
Croatia,190.756763,18.530986
Denmark,573.869221,65.455779
Estonia,80.792742,16.141822
Finland,272.127106,26.381355
France,241.204094,24.606409
Italy,313.079765,28.716678
Lithuania,29.799840,1.124653
Netherlands,454.077698,55.386790


In [70]:
# from df_autonomy_grouped_mean and df_autonomy_grouped_std create coefficient of variation df
df_autonomy_grouped_cv = df_autonomy_grouped_std / df_autonomy_grouped_mean
df_autonomy_grouped_cv

,Frac_counting,impact_P,collab_P,oa_P,TCS,TNCS,P_top1,P_top5,P_top10,P_top50,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
Country,,,,,,,,,,,,,,,,,,,,,
Austria,0.0,0.578307,0.596366,0.596366,0.674639,0.643950,0.713265,0.662601,0.644774,0.603792,...,0.103850,0.453882,0.490636,0.422278,0.331929,0.350372,0.313351,NaN,NaN,NaN
Croatia,0.0,0.868715,0.845434,0.845434,0.858770,0.864828,0.964683,0.823421,0.871912,0.851768,...,0.028832,0.026921,0.094671,0.034191,0.213775,0.267295,0.161275,NaN,NaN,NaN
Denmark,0.0,0.521359,0.484877,0.484877,0.551556,0.547204,0.611004,0.550605,0.563920,0.535060,...,0.145071,0.298264,0.312973,0.283299,0.385789,0.389146,0.381783,NaN,NaN,NaN
Estonia,0.0,0.445205,0.513762,0.513762,0.645975,0.601254,0.875250,0.584501,0.531494,0.543700,...,0.381692,0.544935,0.613304,0.477466,0.298482,0.337298,0.268595,NaN,NaN,NaN
Finland,0.0,0.663374,0.688919,0.688919,0.664942,0.688423,0.692234,0.698463,0.698512,0.686373,...,0.056060,0.416064,0.452260,0.383044,0.349951,0.362005,0.340234,NaN,NaN,NaN
France,0.0,0.709703,0.807218,0.807218,0.800467,0.772020,0.848142,0.827712,0.803696,0.743329,...,0.043381,0.283755,0.301621,0.270693,0.250946,0.261321,0.242354,NaN,NaN,NaN
Italy,0.0,0.700964,0.677370,0.677370,0.692205,0.708944,0.725100,0.715753,0.713187,0.704838,...,0.178797,0.345968,0.360348,0.334910,0.337570,0.354237,0.322127,NaN,NaN,NaN
Lithuania,0.0,0.415438,0.501541,0.501541,0.277530,0.337511,0.193975,0.341541,0.344724,0.369447,...,0.355426,0.639576,0.705020,0.578169,0.268111,0.327391,0.221721,NaN,NaN,NaN
Netherlands,0.0,0.391643,0.438786,0.438786,0.429837,0.424724,0.475664,0.438270,0.429306,0.402688,...,0.082263,0.421332,0.443340,0.398571,0.317228,0.321129,0.313587,NaN,NaN,NaN


In [55]:
# divide 'P_top10' and 'P_top1' by 'impact_P' 
df_autonomy_grouped['PP10'] = df_autonomy_grouped['P_top10'] / df_autonomy_grouped['impact_P']
df_autonomy_grouped['PP1'] = df_autonomy_grouped['P_top1'] / df_autonomy_grouped['impact_P']

In [76]:
df_autonomy_grouped_cv[['TNCS','P_top10', 'P_top1']]

,TNCS,P_top10,P_top1
Country,,,
Austria,0.643950,0.644774,0.713265
Croatia,0.864828,0.871912,0.964683
Denmark,0.547204,0.563920,0.611004
Estonia,0.601254,0.531494,0.875250
Finland,0.688423,0.698512,0.692234
France,0.772020,0.803696,0.848142
Italy,0.708944,0.713187,0.725100
Lithuania,0.337511,0.344724,0.193975
Netherlands,0.424724,0.429306,0.475664


In [2]:
# group df_autonomy by 'Country', calculate sum
df_autonomy_grouped = df_autonomy.groupby('Country').sum()

NameError: name 'df_autonomy' is not defined

In [79]:
df_autonomy_grouped

,Frac_counting,impact_P,collab_P,oa_P,TCS,TNCS,P_top1,P_top5,P_top10,P_top50,...,PP_hybrid_OA_OA_ub,PP_bronze_OA,PP_bronze_OA_lb,PP_bronze_OA_ub,PP_green_OA,PP_green_OA_lb,PP_green_OA_ub,PP_OA_unknown,PP_OA_unknown_lb,PP_OA_unknown_ub
Country,,,,,,,,,,,,,,,,,,,,,
Austria,12,30877.326659,75660.0,75660.0,2.371235e+05,33766.117163,337.433634,1706.754327,3393.655675,16453.813474,...,3.287543,0.795378,0.717059,0.876972,1.372329,1.270999,1.474441,0.0,0.0,0.0
Croatia,2,5986.611323,11517.0,11517.0,3.194233e+04,4988.182768,38.418797,207.625946,437.559517,2692.534282,...,0.133993,0.110920,0.097179,0.125633,0.199399,0.182083,0.218336,0.0,0.0,0.0
Denmark,5,43050.329367,100545.0,100545.0,3.782910e+05,49594.819471,535.641093,2635.441547,5088.218504,24266.017002,...,0.628293,0.472749,0.452271,0.493900,1.291120,1.259515,1.322351,0.0,0.0,0.0
Estonia,2,3598.945009,8013.0,8013.0,2.513510e+04,3357.111869,36.885050,149.604150,304.021451,1637.329006,...,0.217987,0.109692,0.095521,0.124280,0.284211,0.262539,0.306799,0.0,0.0,0.0
Finland,9,32650.975735,77396.0,77396.0,2.469200e+05,35433.795954,342.994126,1714.147940,3506.228107,17797.613219,...,2.110513,0.580024,0.530264,0.630310,2.146574,2.056064,2.236693,0.0,0.0,0.0
France,33,92574.423128,443100.0,443100.0,7.203223e+05,97348.381623,957.400275,4873.172972,9903.910257,48682.581948,...,6.350634,2.984931,2.792103,3.184566,7.901160,7.618063,8.190379,0.0,0.0,0.0
Italy,47,188947.396006,465733.0,465733.0,1.426604e+06,198062.836740,1861.375609,10004.323650,20632.396392,100322.731436,...,4.488942,3.741625,3.461329,4.025905,6.803010,6.447539,7.168939,0.0,0.0,0.0
Lithuania,3,4116.356437,7803.0,7803.0,2.141795e+04,3083.654505,17.393771,106.546469,259.336147,1833.472492,...,0.227000,0.154652,0.130941,0.180043,0.230246,0.199039,0.261209,0.0,0.0,0.0
Netherlands,13,94048.241778,243511.0,243511.0,9.077341e+05,124085.809469,1513.731909,7074.503141,13750.109790,56954.532275,...,4.124454,1.027478,0.976141,1.080858,2.699879,2.617589,2.783221,0.0,0.0,0.0


In [1]:
df_autonomy_grouped.to_excel('leiden-open-ranking-countries-sum.xlsx')

NameError: name 'df_autonomy_grouped' is not defined

In [60]:
df_autonomy_grouped['PP10'] = df_autonomy_grouped['P_top10'] / df_autonomy_grouped['impact_P']
df_autonomy_grouped['PP1'] = df_autonomy_grouped['P_top1'] / df_autonomy_grouped['impact_P']

In [63]:
df_autonomy_grouped[['PP10', 'PP1']]

,PP10,PP1
Country,,
Austria,0.109908,0.010928
Croatia,0.073090,0.006417
Denmark,0.118192,0.012442
Estonia,0.084475,0.010249
Finland,0.107385,0.010505
France,0.106983,0.010342
Italy,0.109197,0.009851
Lithuania,0.063001,0.004226
Netherlands,0.146203,0.016095
